In [20]:
import gspread as gs
from oauth2client.service_account import ServiceAccountCredentials

#구글 스프레드 읽어오기
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive',]
json_file_name = "google spread json.json"

credential = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)

gc = gs.authorize(credential)
spread_url = "https://docs.google.com/spreadsheets/d/1PjKwzw9McKdHvcH42KrmugOXky57xo6HH6OjSw2zABM/edit?resourcekey#gid=1263642264"
doc = gc.open_by_url(spread_url)

worksheet = doc.worksheet("설문지 응답 시트1")
data = worksheet.get_all_values()

import pandas as pd
# 컬럼 이름 지정 할 수 있도록 한다
df = pd.DataFrame(data)
lists = '작성 시각','자취 여부', '불편함', '끼니 해결 방식 1', '끼니 해결 방식 2', '끼니 해결 방식 3', '끼니 해결방식 1순위', '배달/테이크아웃/외식 선호 이유', '레토르트 선호이유', '직접요리 선호이유', '식자재 1', '식자재 2', '식자재 3', '요리 횟수', '요리 선호 시각', '식재료 구입처', '구입처 선호이유', '성별', '출생년도', '자취 기간', '원하는 서비스', '1인가구 아이디어', '번호'
df.columns = lists
df = df.drop(0).reset_index(drop=True).drop(['번호', '작성 시각'], axis=1)

In [21]:
# 응답 확인
df['자취 여부'].unique()

array(['자취를 하고 있다', '자취를 했었다', '없음', '자취4일차...'], dtype=object)

In [22]:
import plotly.express as px
# 응답자 중 자취 유무
df.loc[(df['자취 여부'] == '자취를 하고 있다') | (df['자취 여부'] == '자취를 했었다') | (df['자취 여부'] == '자취4일차...'), '자취 여부'] = 'Y'
df.loc[df['자취 여부'] == '없음', '자취 여부'] = 'N'

live_alone = pd.DataFrame(df['자취 여부'].value_counts())

fig = px.pie(live_alone, values = '자취 여부', names= live_alone.index, hole = .6, title = '1. 현재 자취를 하고 있거나 하신 적이 있습니까?')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    annotations=[dict(text='자취 여부', showarrow=False)])
fig.show()

In [4]:
# 자취 하지 않거나 한 적 없는 응답자 제외
df = df[~df['자취 여부'].str.contains('N')]

count = df['불편함'].value_counts()
# 응답 수 1개인 값 합치기
ect = sum([i for i in count if i == 1])
# 응답이 1건 이상인 데이터만 가져온다
count = count[:len([i for i in count if i > 1])]
unconv = pd.DataFrame(count).reset_index().rename(columns={'불편함':'빈도','index':'불편함'})
unconv.loc[len(count)+1] = ['기타', ect]


In [5]:
reasons = ['식비', '시간 할애', '바쁜 일상', '건강 악화', '맛이 없다', '기타']

# 라벨 줄이기
for idx, val in enumerate(reasons):
    unconv['불편함'].loc[idx] = val

fig = px.pie(unconv, values = '빈도', names= '불편함', labels={unconv['불편함'][0]: 'test'}, hole = .4, title = '2. 자취를 하며 불편함을 가장 많이 느꼈던 부분이 무엇입니까?')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    annotations=[dict(text='불편함', showarrow=False)])
fig.show()


C:\Users\crowdparti\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
eat_way = df.iloc[:, 2:5]
rank_box = []
# 빈도를 계산하고 각각 df를 만든다
for i, pri in zip(eat_way, [1,2,3]):
    pridf = pd.DataFrame(eat_way[i].value_counts())
    pridf['순위'] = pri
    pridf = pridf.reset_index()
    pridf.columns = ['방식','빈도', '순위']
    rank_box.append(pridf)
eat_way = pd.concat(rank_box).reset_index(drop=True)


In [23]:
fig = px.bar(eat_way, x="방식", y="빈도",
             color='순위', barmode='group', text = '빈도', title='3. 자취를 하며 가장 많이 끼니를 해결하신 방식을 선 순위에 따라 세 가지 선택해주세요')
fig.update_traces(textposition='outside')

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()


In [8]:
like_reason = df[['배달/테이크아웃/외식 선호 이유', '레토르트 선호이유', '직접요리 선호이유']]

reason_box = []
for idx, val in enumerate(like_reason):
    reason_box.append(",".join(like_reason.iloc[:, idx][like_reason.iloc[:, idx] != ''].values.tolist()).replace(', ', ',').split(','))

reasons = {'배달/테이크아웃/외식 선호 이유': reason_box[0],'레토르트 선호이유': reason_box[1], '직접요리 선호이유': reason_box[2]}

df_box=[]
for name, data in reasons.items():
    inner = []
    for i in set(data):
        inner.append([i, data.count(i)])
    df_box.append(pd.DataFrame(inner, columns=[f'{name}', '빈도']).sort_values(by='빈도', ascending=False).head())
    


In [29]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

for idx,val in enumerate(df_box):
    fig.add_trace(go.Pie(
            values=df_box[idx].iloc[:, 1], labels= df_box[idx].iloc[:, 0], title=val.columns[0], name=val.columns[0], hole = .6
    ), row=1, col=idx+1)
fig.update_layout(showlegend=False, title='4. 왜 그 끼니 해결 방식을 가장 선호하시나요?')
fig.show()

In [10]:
grocery = df[['식자재 1', '식자재 2', '식자재 3']]
grocery = grocery[grocery['식자재 1'] != '']

In [11]:
grocery_box = []
# 빈도를 계산하고 각각 df를 만든다
for i, pri in zip(grocery, [1,2,3]):
    pridf = pd.DataFrame(grocery[i].value_counts())
    pridf['순위'] = pri
    pridf = pridf.reset_index()
    pridf.columns = ['품목','빈도', '순위']
    grocery_box.append(pridf)
grocery_way = pd.concat(grocery_box).reset_index(drop=True)


In [12]:
fig = px.bar(grocery_way, x="품목", y="빈도",
             color='순위', barmode='group', text = '빈도', title='5-2. 장을 보실 때 어떤 식자재를 주로 구입하시나요?')
fig.update_traces(textposition='outside')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()


In [30]:
place = df[['식재료 구입처', '구입처 선호이유']]
place = place[place['식재료 구입처'] != ''].reset_index(drop=True)
place['test']=''

num = 0    
for i, j, done in place.values:
    num = num+1
    if len(i.split(',')) > 1:
        place = place.drop(index=num)
        for l in range(len(i.split(','))):
            place = place.append([{'식재료 구입처': i.split(',')[l], '구입처 선호이유':j, 'test': 'done'}], ignore_index=True)
    elif len(j.split(',')) > 1:
        place = place.drop(index=num)
        for l in range(len(j.split(','))):
            place = place.append([{'식재료 구입처': i, '구입처 선호이유':j.split(',')[l], 'test': 'done'}], ignore_index=True)
    
            
places = ",".join(place['식재료 구입처'].values.tolist()).replace(' ','').split(',')
places_box = []
for i in set(places):
    places_box.append([i, places.count(i)])
places_df = pd.DataFrame(places_box, columns=['구입처', '빈도']).sort_values(by='빈도', ascending= False).head()


fig = px.pie(places_df, values = '빈도', names= '구입처', labels='구입처', title='5-5. 식재료를 주로 어디에서 구입하시나요?', hole = .4)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [17]:
from hanspell import spell_checker
from eunjeon import Mecab
from tqdm.auto import tqdm
import re
import operator

mecab= Mecab()

# 삭제할 단어
spwords = ['수', '것', '거', '등', '듯']
p = re.compile('[가-힣1-9a-zA-Z]+')

# 빈도 체크
word_dic = {}
whole_box = []
for i in tqdm(df['1인가구 아이디어'].values.tolist()):
    # 철자, 띄어쓰기 검사
    i = spell_checker.check(" ".join(p.findall(i))).checked
    if i != '':
        whole_box.append(i)
        noun = mecab.nouns(i)
        for t in noun:
            if t not in word_dic:
                word_dic[t] = 1
            else:
                word_dic[t] +=1
        try:
            for sp in spwords:
                del word_dic[sp]
        except Exception as e:
            pass

fre = list(sorted(word_dic.items(), key=operator.itemgetter(1), reverse=True))



  0%|          | 0/412 [00:00<?, ?it/s]

In [18]:
import numpy as np
import re

output = []
for f in [ f for f,n in fre if n > 9 ]:
    for val in whole_box:
        val = val.split(' ')
        for i, s in enumerate(val):
            if f in s and len(val[i:i+5]) > 1:
                output.append([f, " ".join(val[i:i+5])])
result = pd.DataFrame(output, columns=['단어', '아이디어'])
result

,단어,아이디어
0,재료,재료 나눔 커뮤니티도 같이 있으면
1,재료,식재료의 가격이 지나치게 높은 것
2,재료,식재료 음식 포장
3,재료,식재료 마트에 있는 새 척
4,재료,식재료 및 식품을 제공하는 플랫폼이
...,...,...
583,플랫폼,플랫폼이 활성화되기를 자취 시작 때부터
584,플랫폼,플랫폼이 있다면 정말 좋을 것
585,플랫폼,플랫폼이 있었으면 좋겠습니다
586,플랫폼,플랫폼이 있으면 좋겠다


In [ ]:
# 유사도 문장 연습

def get_similar(s):
    total = []
    s_pos = mecab.pos(spell_checker.check(s).checked)
    sbox=[]
    for t, a in s_pos:
        if a in ['NNG', 'VV', 'MAG', 'VA']:
            sbox.append(t)

    for i in result['아이디어']:
        o_pos = mecab.pos(i)
        obox=[]
        for t, a in o_pos:
            if a in ['NNG', 'VV', 'MAG', 'VA']:
                obox.append(t)

        sim = len(set(sbox) & set(obox)) / len(set(sbox) | set(obox))
        total.append([i, sim])
        total.sort(key=lambda i:i[1], reverse=True)
    if float(total[0][1]) < 0.45:
        return ('유사 단어가 없습니다.')
    else:
        return ('가장 비슷한 문장은 [{}] 입니다.'.format(total[0][0]))


In [904]:
box={}
final = []
for s in tqdm(result['아이디어']):
    s_pos = mecab.pos(spell_checker.check(s).checked)
    sbox=[]
    for t, a in s_pos:
        if a in ['NNG', 'VV', 'MAG', 'VA']:
            sbox.append(t)
            
    total = []
    for i in result['아이디어']:
        o_pos = mecab.pos(i)
        obox=[]
        for t, a in o_pos:
            if a in ['NNG', 'VV', 'MAG', 'VA']:
                obox.append(t)

        sim = len(set(sbox) & set(obox)) / len(set(sbox) | set(obox))
        total.append([i, sim])
        total.sort(key=lambda i:i[1], reverse=True)
    try:
        if s not in box:
            st = pd.DataFrame(total)[(1.0 > pd.DataFrame(total)[1]) & (pd.DataFrame(total)[1] > 0.45)][0]
            st = ", ".join(list(set(st)))
            box[s] = st

        else:
            box.update([box[s], st])
    except Exception as e:
        pass
    final.append([s, box[s]])

  0%|          | 0/588 [00:00<?, ?it/s]

In [933]:
final_df = pd.DataFrame(final, columns =['기준 문장', '유사 문장'])
len_list=[]
for i in final_df['유사 문장']:
    if i != '':
        len_list.append(len(i.split(', ')))
    else:
        len_list.append(0)
        
final_df['유사 문장 수'] = len_list
final_df.drop_duplicates().sort_values(by='유사 문장 수', ascending = False)[:30]

,기준 문장,유사 문장,유사 문장 수
239,서비스가 있으면 좋겠다,"서비스가 있다면 좋을 것 같다, 키트를 구할 수 있으면 좋을, 배달을 해주는 서비스...",16
241,서비스가 있으면 좋겠어요,"서비스가 있다면 좋을 것 같다, 키트를 구할 수 있으면 좋을, 배달을 해주는 서비스...",16
143,공동구매하는 서비스가 있으면 좋겠어요,"서비스가 있다면 좋을 것 같다, 공동구매를 하고 나눠가질 수 있는, 배달을 해주는 ...",15
145,공동구매해서 나눠갖는 플랫폼 있으면 좋을,"구매할 수 있으면 좋겠다고 매번, 공동구매를 하고 나눠가질 수 있는, 플랫폼 있으면...",12
297,소량으로 저렴하게 구매하고 싶다,"구매를 할 때와 소량 구매를, 소량 구매 가능하거나 공동구매해서 나눠갖는, 소량씩 ...",12
136,배달 비슷한 서비스가 있었으면 좋겠다,"서비스가 있다면 좋을 것 같다, 공동구매하는 서비스가 있으면 좋겠어요, 서비스가 있...",11
129,배달 서비스가 있으면 좋을 것,"서비스가 있다면 좋을 것 같다, 공동구매하는 서비스가 있으면 좋겠어요, 서비스가 있...",11
121,배달을 해주는 서비스가 있으면 좋겠다,"서비스가 있다면 좋을 것 같다, 공동구매하는 서비스가 있으면 좋겠어요, 서비스가 있...",11
242,서비스가 있으면 더 좋음 값이,"서비스가 있다면 좋을 것 같다, 배달을 해주는 서비스가 있으면 좋겠다, 서비스가 있...",10
253,서비스가 있으면 좋을 것 같음,"플랫폼 있으면 좋을 것 같아요, 배달을 해주는 서비스가 있으면 좋겠다, 공동구매하는...",10


In [934]:
whole_box

['굿',
 '없음',
 '우리 동네 재료 나눔 커뮤니티도 같이 있으면 좋겠어요',
 '소포장된 식재료의 가격이 지나치게 높은 것 같다 예 두부 한 모 2 원 두부 반 모 15 원 양파 4개 4 원 양파 한 개 2 원',
 '일회용 쓰레기 없는 식재료 음식 포장',
 '소분되어있는 저렴한 식재료 마트에 있는 새 척 소분 채소는 가격이 지나치게 비싸고 묶음으로 구매하기에는 상하는 경우가 지나치게 많음',
 '전자레인지로 간단 조리 가능한 1인 음식',
 '가까운 거리에 사는 사람과 동일한 제품을 구매하고 싶으면 같이 사서 나눠 가질 수 있는 서비스가 있으면 좋겠다',
 '새벽 배송 시스템이 적용된다면 좋을 것 같아요 기존 새벽 배송을 원활히 진행하고 있는 경우 서비스는 편리하지만 그 판매 상품 가격의 장벽이 자취생이 적극 활용하기에 비교적 높다고 생각됩니다 따라서 제시해주신 대로 대학생 자취생에게 적합한 가격의 식재료 및 식품을 제공하는 플랫폼이 생기고 그 안에서 새벽 배송이 도입된다면 대학생들도 편하게 자유롭게 즐길 수 있는 온라인 식품 쇼핑 문화가 보다 원활하게 확산될 수 있으리라 생각됩니다',
 '배달음식 시켜 먹을 때 양이 많아서 곤란할 때가 있다 요즘은 1인분만 판매하는 가게도 있지만 메뉴가 더 다양해졌으면 좋겠다',
 '장 보러 가는 게 너무 귀찮은데 인터넷으로 시키면 대부분 대량 구매해야 돼서 야채 과일 고기 소량씩 한 묶음으로 판매했으면 좋겠네요',
 '홈플러스에서 1인 가구 맞춤형으로 식재료 팔기 시작했다고 어디선가 봤던 기억이 나는데 정확하게 기억나진 않네요',
 '뭐 하나 해먹으려고 재료 사면 결국 다 처분 못하고 버리게 됩니다 밀키 트나 다양한 소분 재료를 가까운 편의점에서 판매하면 더욱 이용하기 편리할 것 같습니다',
 '여러 사람이 함께 장을 봐서 소분할 수 있는 플랫폼 내지 시스템이 활성화되었으면 좋겠습니다',
 '저렴하게 소량 구매지만 배송비 없이 집 앞까지 배달되면 좋을 거 같아요',
 '직접 요리하기 위해 장을 보면 가격이 너무 

In [932]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

재료
요리
배달
구매
인
서비스
음식
소량
가구
때
식
가격
키트
인분
공동
배송
음식물
쓰레기
판매
생각
소분
사람
플랫폼


In [894]:
set(['test'])

{'test'}

In [862]:
box[s]

{'공동구매 그런 플랫폼 만들어도 좋을', '플랫폼 있으면 좋을 것 같아요'}